In [1]:
#Import all the tools we'll need
import pandas as pd
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint
import matplotlib.pyplot as plt
import functools

C:\Users\Zade Alfalah\AppData\Local\conda\conda\envs\py37\lib\site-packages\dask\config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
#because of update to p4tools, 6/27/19
from p4tools import io

In [3]:
#because of update to p4tools
tiles = io.get_tile_coordinates(update=True)

In [4]:
tiles.head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
0,1,1,420.0,324.0,-85.504144,-85.556701,104.952104,-68.380674,256.056147,-3370.636698,APF0000cwz,ESP_012079_0945
1,1,2,420.0,872.0,-85.502300,-85.554878,104.934805,-68.331337,256.181661,-3370.628630,APF0000ck9,ESP_012079_0945
2,1,3,420.0,1420.0,-85.500454,-85.553054,104.917486,-68.281853,256.307249,-3370.620295,APF0000cty,ESP_012079_0945
3,1,4,420.0,1968.0,-85.498607,-85.551229,104.900198,-68.232522,256.433112,-3370.615690,APF0000ciy,ESP_012079_0945
4,1,5,420.0,2516.0,-85.496762,-85.549404,104.882968,-68.183281,256.558509,-3370.606455,APF0000cwp,ESP_012079_0945


In [5]:
tiles[tiles['obsid'] == 'ESP_012884_0935' ].head()
#check the tile_coordinates csv, see no X,Y,Z bodyfixed coords.

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
4777,1,1,420.0,324.0,-86.455091,-86.496595,NaN,NaN,NaN,NaN,APF0000dsd,ESP_012884_0935
4778,1,2,420.0,872.0,-86.452087,-86.493626,NaN,NaN,NaN,NaN,APF0000dqa,ESP_012884_0935
4779,1,3,420.0,1420.0,-86.449083,-86.490657,NaN,NaN,NaN,NaN,APF0000dru,ESP_012884_0935
4780,1,4,420.0,1968.0,-86.446077,-86.487686,NaN,NaN,NaN,NaN,APF0000dpx,ESP_012884_0935
4781,1,5,420.0,2516.0,-86.443066,-86.484709,NaN,NaN,NaN,NaN,APF0000dsb,ESP_012884_0935


In [10]:
olddatafull = pd.read_csv("n_obj_df.csv") #old dataframe we're comparing to
#updating n_obj_df (from calculate_fractional_coverage_new notebook)
olddata=olddatafull[olddatafull['MY'] == 29] #we only want to look at MY29

In [13]:
allregions = olddata['region_name'].unique()

In [6]:
#Michael's code, gets the polygon of an obsid
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [7]:
def oneinter(pA, pB): #One intersection between polygon A and polygon B
    oneint = pA.intersection(pB)
    return oneint

In [14]:
#Now with reduce, self made function:
def createreducedf(polygon_array):
    """
    
    """
    #This block below gets our combinations of polygons, works as intended
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end
    
    
    for ele in combos:
        d = {}
        #create a blank dictionary
        bucket = []
        #create an empty bucket
        for number in ele:
            #where number would be (for example) polygon0
            bucket.append(polygon_array[number])
        #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = bucket
        #give the dictionary the polygons used
        Multi = MultiPolygon(bucket)
        #create a multipolygon out of the polygons used
        d['multipolygon'] = Multi
        #create a dictionary input of the overall multipolygon
        if len(bucket) == 1:
             d['combo_area'] = bucket[0].area
        else:
            r = functools.reduce(oneinter, bucket)

            d['combo_area'] = r.area
            
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [16]:
for region in allregions:
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d = {}
        d['region_name'] = region
        d['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d['Obsid_Area'] = area
        d['polygon_shape'] = x
        bucket.append(d)
    df1 = pd.DataFrame(bucket)
    df3 = reducechecker(df1.polygon_shape)
    bucket2 = []
    for ele in df3.index:
        #print(ele)
        polylist = df3.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.Obsid_Used[df1['polygon_shape'] == poly].values[0]
            bucket.append(x)
        bucket2.append(bucket)
    df3['obsids_used'] = np.array(bucket2)
    for i in range(len(df3.elements_used)): 
    df3.at[i, 'number_hirise'] = int(len(df3.elements_used[i]))
    bucket2 = []
    for obslist in reducedf.obsids_used:
        bucket = []
        for obs in obslist:
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    df3['Ls_used'] = np.array(bucket2)
    df3['region_name'] = region
    df3 = df3.drop(['elements_used'], axis = 1) 
    reducedf.to_csv(f"{region}_combos.csv", index=False)

IndentationError: expected an indented block (<ipython-input-16-74fe428648a9>, line 25)